# Hacktiv8 Phase 0: Graded Challenge 3

---

Graded Challenge ini dibuat guna mengevaluasi pembelajaran pada Hacktiv8 Data Science Fulltime Program khususnya pada Practical Statistics.

## Introduction

By [Rifky Aliffa](https://github.com/Penzragon)

![Kickstarter](https://i.imgur.com/mN5a9eh.png)

### Dataset

Data yang digunakan adalah dataset yang berisi project yang dikumpulkan dari [Kickstarter Platform](https://www.kickstarter.com/). Dataset dapat dilihat di [Kaggle](https://www.kaggle.com/kemical/kickstarter-projects?select=ks-projects-201801.csv).

## Import Library

Mengimport semua library yang akan digunakan, dalam project ini menggunakan **5** library yaitu **Pandas**, **NumPy**, **SciPy**, **Matplotlib**, dan **Seaborn**.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns